In [1]:
# >>>>>>>>>>>>>>>>>>>>>>>>>>>>>> imports >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
!pip install inflect
!pip install contractions 
!pip install krovetzstemmer
import contractions
from contractionsOne import CONTRACTION_MAP

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.stem.lancaster import LancasterStemmer
from krovetzstemmer import Stemmer
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('stopwords')


import inflect
import numpy as np
import pandas as pd
import sys, setuptools, tokenize
import re, unicodedata, string  
from bs4 import BeautifulSoup

     |████████████████████████████████| 286kB 6.5MB/s 
     |████████████████████████████████| 327kB 9.2MB/s 
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.2-cp37-cp37m-linux_x86_64.whl size=85400 sha256=31fbf94344056739b723f0f5432efd55931fd6b09324e1734b08fe77ac3c60b3
  Stored in directory: /root/.cache/pip/wheels/3a/03/34/77e3ece0bba8b86bfac88a79f923b36d805cad63caeba38842
Successfully built pyahocorasick
     |████████████████████████████████| 122kB 6.9MB/s 
  Created wheel for krovetzstemmer: filename=KrovetzStemmer-0.8-cp37-cp37m-linux_x86_64.whl size=352795 sha256=9768031a0a37676d8b30569fa3f05a96dd2b690f6324d0b19ddb494d81f54d6c
  Stored in directory: /root/.cache/pip/wheels/8b/6d/eb/06535521f7a7c68ff74257af8547f61661210851d732ed7352
Successfully built krovetzstemmer
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/p

In [2]:
# >>>>>>>>>>>>>>>>>>>>>>>>>>>>>> loading the dataset >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

pd.options.mode.chained_assignment = None
df = pd.read_csv('/content/testing.csv')
review_df = df
review_df

,reviewText
0,<h1>I've been using these for over a year now....
1,NaN
2,https://www.hi.com I have owned these for over...
3,I'll be there within 5 min. Shouldn't you be t...
4,I ** [bought] these thinking they were the tic...
5,#Work Good. I do not have another reference. Y...
6,भारत Not as immersive and noise-cancelling as ...
7,DINUVI I own these and the Bose Quiet Comfort...
8,�� I bought these at the flea market for 10 do...
9,Let's make it short story. It extremly difficu...


In [3]:
# >>>>>>>>>>>>>>>>>>>>>>>>>>>>>> removing null values >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

#removing null values
# counting rows after removing null values
# df.dropna(subset=['reviewText'], inplace=True)
# df

In [4]:
# >>>>>>>>>>>>>>>>>>>>>>>>>>>>>> removing URL >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

def remove_URL(reviewText):
    url = re.compile(r'https?://\S+|www\.\S+')
    return url.sub(r'', str(reviewText))
df['reviewText']=df['reviewText'].apply(remove_URL)
df.head(10)

,reviewText
0,<h1>I've been using these for over a year now....
1,nan
2,I have owned these for over two years and wor...
3,I'll be there within 5 min. Shouldn't you be t...
4,I ** [bought] these thinking they were the tic...
5,#Work Good. I do not have another reference. Y...
6,भारत Not as immersive and noise-cancelling as ...
7,DINUVI I own these and the Bose Quiet Comfort...
8,�� I bought these at the flea market for 10 do...
9,Let's make it short story. It extremly difficu...


In [5]:
# >>>>>>>>>>>>>>>>>>>>>>>>>>>>>> removing HTML tags >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

def remove_html(reviewText):
    html=re.compile(r'<.*?>')
    return html.sub(r'',str(reviewText))
df['reviewText'] = df['reviewText'].apply(remove_html)
df.head(10)

,reviewText
0,I've been using these for over a year now.
1,nan
2,I have owned these for over two years and wor...
3,I'll be there within 5 min. Shouldn't you be t...
4,I ** [bought] these thinking they were the tic...
5,#Work Good. I do not have another reference. Y...
6,भारत Not as immersive and noise-cancelling as ...
7,DINUVI I own these and the Bose Quiet Comfort...
8,�� I bought these at the flea market for 10 do...
9,Let's make it short story. It extremly difficu...


In [6]:
# >>>>>>>>>>>>>>>>>>>>>>>>>>>>>> removing square brackets and the inside of the square bracket >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

def remove_between_square_brackets(reviewText):
    return re.sub('\[[^]]*\]', '', str(reviewText))
df['reviewText'] = df['reviewText'].apply(remove_between_square_brackets)
df.head(10)

,reviewText
0,I've been using these for over a year now.
1,nan
2,I have owned these for over two years and wor...
3,I'll be there within 5 min. Shouldn't you be t...
4,I ** these thinking they were the ticket to n...
5,#Work Good. I do not have another reference. Y...
6,भारत Not as immersive and noise-cancelling as ...
7,DINUVI I own these and the Bose Quiet Comfort...
8,�� I bought these at the flea market for 10 do...
9,Let's make it short story. It extremly difficu...


In [7]:
# >>>>>>>>>>>>>>>>>>>>>>>>>>>>>> removing Pictures/Tags/Symbols/Emojis >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

def remove_emoji(reviewText):
    emoj = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                      "]+", re.UNICODE)
    return re.sub(emoj, '', str(reviewText))

df['reviewText'] = df['reviewText'].apply(remove_emoji)
df.head(10)

,reviewText
0,I've been using these for over a year now.
1,nan
2,I have owned these for over two years and wor...
3,I'll be there within 5 min. Shouldn't you be t...
4,I ** these thinking they were the ticket to n...
5,#Work Good. I do not have another reference. Y...
6,भारत Not as immersive and noise-cancelling as ...
7,DINUVI I own these and the Bose Quiet Comfort...
8,I bought these at the flea market for 10 doll...
9,Let's make it short story. It extremly difficu...


In [8]:
# >>>>>>>>>>>>>>>>>>>>>>>>>>>>>> removing non ascii >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

df['reviewText'] = [w.encode("ascii", "ignore").decode() for w in df['reviewText']]
df

,reviewText
0,I've been using these for over a year now.
1,nan
2,I have owned these for over two years and wor...
3,I'll be there within 5 min. Shouldn't you be t...
4,I ** these thinking they were the ticket to n...
5,#Work Good. I do not have another reference. Y...
6,Not as immersive and noise-cancelling as othe...
7,DINUVI I own these and the Bose Quiet Comfort...
8,I bought these at the flea market for 10 doll...
9,Let's make it short story. It extremly difficu...


In [9]:
# >>>>>>>>>>>>>>>>>>>>>>>>>>>>>> convert to lower case >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

df['reviewText'] = [w.lower() for w in df['reviewText']]
df

,reviewText
0,i've been using these for over a year now.
1,nan
2,i have owned these for over two years and wor...
3,i'll be there within 5 min. shouldn't you be t...
4,i ** these thinking they were the ticket to n...
5,#work good. i do not have another reference. y...
6,not as immersive and noise-cancelling as othe...
7,dinuvi i own these and the bose quiet comfort...
8,i bought these at the flea market for 10 doll...
9,let's make it short story. it extremly difficu...


In [10]:
# >>>>>>>>>>>>>>>>>>>>>>>>>>>>>> expand contractions >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

def expand_contractions(reviewText):
    contractionsPattern = re.compile('({})'.format('|'.join(CONTRACTION_MAP.keys())),flags=re.IGNORECASE|re.DOTALL)
    def expand_match(contraction):
        match = contraction.group(0)
        firstChar = match[0]
        expandedContraction = CONTRACTION_MAP.get(match) \
            if CONTRACTION_MAP.get(match) \
            else CONTRACTION_MAP.get(match.lower())
        expandedContraction = firstChar+expandedContraction[1:]
        return expandedContraction
    reviewText = contractionsPattern.sub(expand_match, str(reviewText))
    reviewText = re.sub("'", "", reviewText)
    return reviewText

df['reviewText']=df['reviewText'].apply(expand_contractions)
df

,reviewText
0,i have been using these for over a year now.
1,nan
2,i have owned these for over two years and wor...
3,i will be there within 5 min. should not you b...
4,i ** these thinking they were the ticket to n...
5,#work good. i do not have another reference. y...
6,not as immersive and noise-cancelling as othe...
7,dinuvi i own these and the bose quiet comfort...
8,i bought these at the flea market for 10 doll...
9,let us make it short story. it extremly diffic...


In [11]:
# >>>>>>>>>>>>>>>>>>>>>>>>>>>>>> removing all punctuations >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

import string
def clear_punctuation(s):
  clear_string = ""
  for symbol in s:
    if symbol not in string.punctuation:
      clear_string += symbol
  return clear_string
df['reviewText'] = df['reviewText'].apply(clear_punctuation)
df.head(10)
# print(clear_punctuation(df))

,reviewText
0,i have been using these for over a year now
1,nan
2,i have owned these for over two years and wor...
3,i will be there within 5 min should not you be...
4,i these thinking they were the ticket to noi...
5,work good i do not have another reference you ...
6,not as immersive and noisecancelling as other...
7,dinuvi i own these and the bose quiet comfort...
8,i bought these at the flea market for 10 doll...
9,let us make it short story it extremly difficu...


In [12]:
# >>>>>>>>>>>>>>>>>>>>>>>>>>>>>> tokenizing sentence >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

sentence_all = []
for index, row in df.iterrows():
  sentences_with_feature = []
  for sen in sent_tokenize(row['reviewText']):
    sentences_with_feature.append(sen)
  sentence_all.append(sentences_with_feature)

list_of_tuples = list(zip(sentence_all)) 
df_sentences = pd.DataFrame(list_of_tuples,columns = ['reviewText'])
df_sentences

,reviewText
0,[i have been using these for over a year now]
1,[nan]
2,[ i have owned these for over two years and wo...
3,[i will be there within 5 min should not you b...
4,[i these thinking they were the ticket to no...
5,[work good i do not have another reference you...
6,[ not as immersive and noisecancelling as othe...
7,[dinuvi i own these and the bose quiet comfor...
8,[ i bought these at the flea market for 10 dol...
9,[let us make it short story it extremly diffic...


In [13]:
# >>>>>>>>>>>>>>>>>>>>>>>>>>>>>> tokenizing words >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

def wordTokenize(sentence):
    tokens = [w for t in (sentence.apply(word_tokenize)) for w in t]
    return tokens

In [14]:
# >>>>>>>>>>>>>>>>>>>>>>>>>>>>>> replacing numbers with string  >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

def stringNumbers(sentence):
  p = inflect.engine()
  stringNumbers = []
  for word in sentence:
    if word.isdigit():
      # new_word = num2word.to_card(15)
      new_word = p.number_to_words(word)
      stringNumbers.append(new_word)
      
    else:
      stringNumbers.append(word)
  # stringNumbers = [w.lower() for w in removedPunctuation]
  return stringNumbers

In [15]:
# >>>>>>>>>>>>>>>>>>>>>>>>>>>>>> removing stopwords >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

from nltk.corpus import stopwords
def stopWords(sentence):
  stop_words = set(stopwords.words('english'))
  removedStopwords = [word for word in sentence if not word in stop_words]
  return removedStopwords

In [16]:
# >>>>>>>>>>>>>>>>>>>>>>>>>>>>>> stemming words >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

def stemmers(sentence):
  #stemmer = PorterStemmer()
  # stemmer = LancasterStemmer()
  stemmer = Stemmer()
  stems = []
  for word in sentence:
      stem = stemmer.stem(word)
      stems.append(stem)
  return stems

In [17]:
# >>>>>>>>>>>>>>>>>>>>>>>>>>>>>> lemmatize_verbs >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

def lemmatizers(sentence):
  lemmatizer = WordNetLemmatizer()
  lemmas = []
  for word in sentence:
      lemma = lemmatizer.lemmatize(word, pos='v')
      lemmas.append(lemma)
  return lemmas

In [18]:
from nltk.tokenize import word_tokenize
alltokens = []
wordTokens = pd.DataFrame()
for index, row in df_sentences.iterrows():
  list_of_tuples = list(zip(row['reviewText'])) 
  df_sentences_w = pd.DataFrame(list_of_tuples,columns = ['reviewText'])
  
  wordTokens = wordTokenize(df_sentences_w['reviewText'])
  stringNumber = stringNumbers(wordTokens)
  stopword = stopWords(stringNumber)
  stemmer = stemmers(stopword)
  lemmatizer = lemmatizers(stemmer)
  alltokens.append(lemmatizer)
alltokens

[['use', 'year'],
 ['nan'],
 ['own', 'two', 'years', 'wear', 'thirty', 'time', 'mostly', 'plane', 'ride'],
 ['within', 'five', 'min'],
 ['think', 'ticket', 'noisefree', 'pleasant', 'air', 'travel'],
 ['work', 'good', 'another', 'reference', 'get', 'tire', 'wear'],
 ['immerse',
  'noisecancell',
  'expensive',
  'model',
  'satisfy',
  'value',
  'would',
  'absolutely',
  'buy'],
 ['dinuvi',
  'bose',
  'quiet',
  'comfort',
  'two',
  'twenty-five thousand',
  'honestly',
  'tell',
  'bose',
  'phone',
  'impressive',
  'better',
  'certainly',
  'worth',
  'twenty-five thousand'],
 ['buy',
  'flea',
  'market',
  'ten',
  'dollar',
  'love',
  'ever',
  'since',
  'sound',
  'great',
  'great',
  'range',
  'two hundred thousand, three hundred',
  'feet',
  'like',
  'say',
  'manual',
  'battery',
  'still',
  'hold',
  'charge',
  'great',
  'pay'],
 ['let',
  'us',
  'make',
  'short',
  'story',
  'extrem',
  'difficult',
  'find',
  'setup',
  'sound',
  'clear',
  'work',
  'ex

In [19]:
# >>>>>>>>>>>>>>>>>>>>>>>>>>>>>> de-tokenise >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

from nltk.tokenize.treebank import TreebankWordDetokenizer
from nltk.tokenize import word_tokenize

detokenizeall = []
for alltokensRow in alltokens:
  reviewWordDetokenize = TreebankWordDetokenizer().detokenize(alltokensRow)
  detokenizeall.append(reviewWordDetokenize)

review_df['cleaned'] = detokenizeall
review_df

,reviewText,cleaned
0,i have been using these for over a year now,use year
1,nan,nan
2,i have owned these for over two years and wor...,own two years wear thirty time mostly plane ride
3,i will be there within 5 min should not you be...,within five min
4,i these thinking they were the ticket to noi...,think ticket noisefree pleasant air travel
5,work good i do not have another reference you ...,work good another reference get tire wear
6,not as immersive and noisecancelling as other...,immerse noisecancell expensive model satisfy v...
7,dinuvi i own these and the bose quiet comfort...,dinuvi bose quiet comfort two twenty-five thou...
8,i bought these at the flea market for 10 doll...,buy flea market ten dollar love ever since sou...
9,let us make it short story it extremly difficu...,let us make short story extrem difficult find ...


In [21]:
# >>>>>>>>>>>>>>>>>>>>>>>>>>>>>> saving the cleaned data >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

review_df.to_csv('testing_cleaned_with_lemmatized.csv') 
review_df.to_csv('testing_cleaned_without_lemmatized.csv') 
review_df.to_csv('testing_cleaned_without_stemmer.csv') 
review_df.to_csv('testing_cleaned_with_stemmer.csv') 

review_df

,reviewText,cleaned
0,i have been using these for over a year now,use year
1,nan,nan
2,i have owned these for over two years and wor...,own two years wear thirty time mostly plane ride
3,i will be there within 5 min should not you be...,within five min
4,i these thinking they were the ticket to noi...,think ticket noisefree pleasant air travel
5,work good i do not have another reference you ...,work good another reference get tire wear
6,not as immersive and noisecancelling as other...,immerse noisecancell expensive model satisfy v...
7,dinuvi i own these and the bose quiet comfort...,dinuvi bose quiet comfort two twenty-five thou...
8,i bought these at the flea market for 10 doll...,buy flea market ten dollar love ever since sou...
9,let us make it short story it extremly difficu...,let us make short story extrem difficult find ...
